## 第2章 最小二乗法

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
N = 10
M = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
"""
データセットの作成
"""
def create_dataset(n):
    dataset = pd.DataFrame(columns={'x', 'y'})
    for i in range(n):
        x = float(i)/float(n - 1)
        y = np.sin(2 * np.pi * x) + np.random.normal(scale=0.3)
        dataset = dataset.append(pd.Series([x, y], index=['x', 'y']), ignore_index=True)

    return dataset

In [ ]:
dataset = create_dataset(10)
sns.scatterplot(data=dataset, x='x', y='y')
print(dataset)

分析対象のトレーニングセット$\{(x_n, t_n)\}_{n=1}^{N}$が与えられたとするときに。
目的変数$t$を推測するために求める$M$次多項式$f(x)$を決定する。各次数の係数を$\{w_m\}_{m=1}^M$を求める。
$$
 f(x) = \sum_{m=0}^{M}w_mx^m
$$

係数を求める指標として、次式の二乗誤差を最小にするという条件を持ちる。
$$
E_D = \frac{1}{2} \sum_{n=1}^N\{f(x_n)-t_n\}^2
$$

係数の個数$M+1$がトレーニングセットの個数$N$以上であれば、つまり$M + 1 \le N$であれば、係数は次式で求められる。
$$
\boldsymbol{w} = (\Phi^\top\Phi)^{-1}\Phi^\top t
$$

In [ ]:
# datasetの関数fに対する最小二乗誤差を求める
def rms_error(dataset, f):
    error = 0.0
    for i, d in dataset.iterrows():
        x, y = d.x, d.y
        error += 0.5 * (y - f(x))**2
    
    return np.sqrt(2 * error / len(dataset))

In [ ]:
"""
最小二乗法で解を求める
dataeset: 
m: 次数
"""
def resolve(dataset, m):
    t = dataset.y
    phi = DataFrame()
    for i in range(0, m + 1):
        p =dataset.x**i
        phi = pd.concat([phi, p], axis=1)
    tmp = np.linalg.inv(np.dot(phi.T, phi))
    ws = np.dot(np.dot(tmp, phi.T), t)

    def f(x):
        y = 0
        for i, w in enumerate(ws):
            y += w * (x**i)
        return y
    
    return (f, ws)


In [ ]:
train_set = create_dataset(N)
test_set = create_dataset(N)
df_ws = DataFrame()

for c, m in enumerate(M):
    sns.scatterplot(data=train_set, x='x', y='y')
    f, ws = resolve(train_set, m)
    df_ws = df_ws.append(Series(ws,name="M=%d" % m))

    # 正解と予測曲線を表示
    line = DataFrame()
    line['x'] = np.linspace(0, 1, 101)
    line['t'] = np.sin(2 * np.pi * line['x'])
    line['y'] = f(line['x'])
    t = sns.lineplot(data=line, x='x', y='t')
    t.lines[0].set_linestyle('--')
    sns.lineplot(data=line, x='x', y='y')
    plt.show()

print(df_ws)

In [ ]:
"""
トレーニングセットとテストセットでRMSを求める
"""
rms_df = DataFrame(columns=['Train', 'Test'])
for m in range(0, 10):
    f, ws = resolve(train_set, m)
    train_error = rms_error(train_set, f)
    test_error = rms_error(test_set, f)
    rms_df = rms_df.append(Series([train_error, test_error], index=['Train', 'Test']), ignore_index=True)

print(rms_df)